In [1]:
#connect with drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/MyDrive/Bo_RA/

/content/gdrive/MyDrive/Bo_RA


In [3]:
import pandas as pd
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

In [4]:
t1_unit = pd.read_stata('top_units_US_unified_2020_10k.dta')[['app_id', 'platform', 'app_id_unified']]
t1_revenue = pd.read_stata('top_revenue_US_unified_2020_10k.dta')[['app_id', 'platform', 'app_id_unified']]

In [5]:
and_t1_unit = set(t1_unit[t1_unit['platform']=='android']['app_id']) #8719
ios_t1_unit = set(t1_unit[t1_unit['platform']=='ios']['app_id']) #9327
and_t1_revenue = set(t1_revenue[t1_revenue['platform']=='android']['app_id']) #7530
ios_t1_revenue = set(t1_revenue[t1_revenue['platform']=='ios']['app_id']) #10026

In [6]:
len(set(pd.read_stata('top_revenue_US_unified_2020_10k.dta')['app_id_unified']))#check

10000

In [7]:
app_t2_unit = set(pd.read_stata('top_units_US_panel_2018-2021_tsfilled.dta')['app_id_unified'])#6518
t1_unit['is_t2'] = t1_unit['app_id_unified'].apply(lambda x: 1 if x in app_t2_unit else 0)
ios_t2_unit = t1_unit.loc[(t1_unit['is_t2'] == 1)&(t1_unit['platform']=='ios')]['app_id'] #7330
and_t2_unit = t1_unit.loc[(t1_unit['is_t2'] == 1)&(t1_unit['platform']=='android')]['app_id'] #6951

In [8]:
app_t2_revenue = set(pd.read_stata('top_revenue_US_panel_2018-2021_tsfilled.dta')['app_id_unified'])#6032
t1_revenue['is_t2'] = t1_revenue['app_id_unified'].apply(lambda x: 1 if x in app_t2_revenue else 0)
ios_t2_revenue = t1_revenue.loc[(t1_revenue['is_t2'] == 1)&(t1_revenue['platform']=='ios')]['app_id'] #6733
and_t2_revenue = t1_revenue.loc[(t1_revenue['is_t2'] == 1)&(t1_revenue['platform']=='android')]['app_id'] #6484

In [9]:
ios_df = pd.read_pickle('ios_app.pickle')[['app_id', 'version', 'description', 'date',
                 'len_word', 'len_char', 'update_by_num', 'len_char_ttl',
                 'para', 'update_by_direct_mention', 
                 'privacy_mention', 'update_by_feature_bug',
                 'update_by_num_modified', 'update_by_para', 'system_mention']]

In [10]:
ios_update = set(ios_df['app_id'])

In [11]:
and_df = pd.read_pickle('android_app.pickle')[['app_id', 'version', 'description', 'date',
                 'len_word', 'len_char', 'update_by_num', 'len_char_ttl',
                 'para', 'update_by_direct_mention', 
                 'privacy_mention', 'update_by_feature_bug',
                 'update_by_num_modified', 'update_by_para', 'system_mention']]
and_update = set(and_df['app_id'])

1. Check if all apps in the type 1 lists could be found in the version update samples (the ones you previously used).

In [12]:
#percentage of apps in the version update sample
print(sum(t1_unit['app_id'].apply(lambda x: 1 if x in ios_update else 0))/len(ios_t1_unit)) #7784 / 9327
print(sum(t1_revenue['app_id'].apply(lambda x: 1 if x in ios_update else 0))/len(ios_t1_revenue)) #5877 / 10026

0.8345663128551517
0.5861759425493717


In [13]:
print(sum(t1_unit['app_id'].apply(lambda x: 1 if x in and_update else 0))/len(and_t1_unit)) #7693 / 8719
print(sum(t1_revenue['app_id'].apply(lambda x: 1 if x in and_update else 0))/len(and_t1_revenue)) #4449 / 7530

0.8823259548113316
0.5908366533864542


2. (Requested by Huan) Generate summary statistic tables at the app level for type 1 lists, respectively. Include as many variables as you have created and make sense to perform summary statistics. E.g., whether they appear in the version updates records, number of paragraphs/words/characters, number of updates (per month or total), number of major updates or bug fixes (based on the multiple measures you constructed), etc. This gives an overview for Bo and Huan, who have not closely followed the details. For apps with multiple version updates, take the average/sum/dummy or whatever makes sense for each app, then generate summary statistics. 

In [71]:
ios_df['is_t1_unit'] = ios_df['app_id'].apply(lambda x: 1 if x in ios_t1_unit else 0)
ios_df['is_t1_revenue'] = ios_df['app_id'].apply(lambda x: 1 if x in ios_t1_revenue else 0)
and_df['is_t1_unit'] = and_df['app_id'].apply(lambda x: 1 if x in and_t1_unit else 0)
and_df['is_t1_revenue'] = and_df['app_id'].apply(lambda x: 1 if x in and_t1_revenue else 0)

In [89]:
def gen_summary(update_df, platform = 'ios', sort = 'unit'):
    temp_df =  update_df[update_df['is_t1_{}'.format(sort)] == 1]
    temp_sum = pd.DataFrame()
    temp_sum['ttl_update'] = temp_df.groupby('app_id').count()['version']
    temp_sum = temp_sum.reset_index()
    #print(temp_sum.head())

    temp_sum = \
    temp_df.groupby('app_id').sum().reset_index()[['app_id', 'update_by_direct_mention', 
                                                'update_by_feature_bug', 
                                                'update_by_num_modified', 
                                                'update_by_para']].merge(temp_sum, on = 'app_id')
    #print(temp_sum.head())

    update_monthly = pd.read_csv('{}_num_update_app_month.csv'.format(platform))[['app_id','ym', 'num_update']].\
    groupby('app_id').mean().reset_index()
    #print(update_monthly.head())
    update_monthly['app_id'] = update_monthly['app_id'].apply(lambda x: str(x))
    update_monthly.columns = ['app_id', 'avg_monthly_update']
    temp_sum = pd.merge(temp_sum, update_monthly, how = 'inner', on = 'app_id')
    #print(temp_sum.head())

    temp_sum = \
    temp_df.groupby('app_id').mean().reset_index()[['app_id', 'len_word', 
                                                    'len_char', 'para', 
                                                    'len_char_ttl']].merge(temp_sum, on = 'app_id')

    temp_sum = temp_sum.rename(columns={'len_word': 'avg_len_word', 
                                        'len_char': 'avg_len_char', 
                                        'para': 'avg_para',
                                        'len_char_ttl': 'avg_len_char_ttl'})
    print('*******Summary statistics for top {} apps sorted by {}***********'.format(platform, sort))
    return temp_sum.describe()

In [77]:
gen_summary(ios_df, platform = 'ios', sort = 'unit')

*******Summary statistics for top ios apps sorted by unit***********


,avg_len_word,avg_len_char,avg_para,avg_len_char_ttl,update_by_direct_mention,update_by_feature_bug,update_by_num_modified,update_by_para,ttl_update,avg_monthly_update
count,7784.000000,7784.000000,7784.000000,7784.000000,7784.000000,7784.000000,7784.000000,7784.000000,7784.000000,7784.000000
mean,28.267687,140.512117,3.398497,176.154016,0.619476,3.894784,3.643499,5.403777,73.574769,1.239354
std,28.783998,139.794113,2.875070,176.226819,2.702255,7.983978,10.824519,5.733065,69.930220,0.964344
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.027778
25%,9.500000,50.590474,1.678925,63.235122,0.000000,0.000000,0.000000,1.000000,22.000000,0.547619
50%,21.000000,105.925134,2.571429,132.672619,0.000000,1.000000,2.000000,3.000000,50.500000,1.000000
75%,37.566646,184.824519,4.102785,231.405594,0.000000,5.000000,4.000000,8.000000,104.000000,1.653561
max,391.272727,1970.220779,46.794872,2463.987013,67.000000,167.000000,271.000000,42.000000,504.000000,10.000000


In [78]:
gen_summary(ios_df, platform = 'ios', sort = 'revenue')

*******Summary statistics for top ios apps sorted by revenue***********


,avg_len_word,avg_len_char,avg_para,avg_len_char_ttl,update_by_direct_mention,update_by_feature_bug,update_by_num_modified,update_by_para,ttl_update,avg_monthly_update
count,5877.000000,5877.000000,5877.000000,5877.000000,5877.000000,5877.000000,5877.000000,5877.000000,5877.000000,5877.000000
mean,34.526427,171.718325,4.120015,215.711474,0.792411,3.927854,3.410413,5.578697,74.785605,1.319751
std,35.539291,172.595351,3.439474,217.648894,2.963091,6.928442,9.754213,5.530238,66.189282,0.921360
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.027778
25%,12.256684,64.528302,1.977273,80.411765,0.000000,0.000000,0.000000,2.000000,27.000000,0.666667
50%,26.212121,131.025000,3.162791,163.875000,0.000000,2.000000,2.000000,4.000000,56.000000,1.105882
75%,44.851351,219.285714,5.139706,275.581395,0.000000,5.000000,4.000000,8.000000,104.000000,1.736842
max,391.272727,1970.220779,41.441718,2463.987013,67.000000,129.000000,271.000000,42.000000,504.000000,9.975610


In [91]:
gen_summary(and_df, platform = 'android', sort = 'unit')

*******Summary statistics for top android apps sorted by unit***********


,avg_len_word,avg_len_char,avg_para,avg_len_char_ttl,update_by_direct_mention,update_by_feature_bug,update_by_num_modified,update_by_para,ttl_update,avg_monthly_update
count,7693.000000,7693.000000,7693.000000,7693.000000,7693.000000,7693.000000,7693.000000,7693.000000,7693.000000,7693.000000
mean,28.376855,146.234196,3.053162,181.948711,0.362017,7.000260,4.710776,4.089432,68.994540,1.138709
std,28.397758,162.477186,2.274253,199.325210,1.829824,15.427164,11.396425,5.254378,68.609391,0.882506
min,0.000000,0.000000,0.285714,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.027397
25%,8.000000,41.815385,1.404255,51.500000,0.000000,0.000000,1.000000,0.000000,21.000000,0.512821
50%,21.925926,109.200000,2.400000,136.559322,0.000000,2.000000,2.000000,2.000000,47.000000,0.920000
75%,40.809524,202.216667,3.960317,252.904762,0.000000,8.000000,5.000000,6.000000,94.000000,1.520408
max,433.533333,4153.500000,38.791667,4868.625000,40.000000,287.000000,238.000000,43.000000,694.000000,11.488372


In [90]:
gen_summary(and_df, platform = 'android', sort = 'revenue')

*******Summary statistics for top android apps sorted by revenue***********


,avg_len_word,avg_len_char,avg_para,avg_len_char_ttl,update_by_direct_mention,update_by_feature_bug,update_by_num_modified,update_by_para,ttl_update,avg_monthly_update
count,4449.000000,4449.000000,4449.000000,4449.000000,4449.000000,4449.000000,4449.000000,4449.000000,4449.000000,4449.000000
mean,34.999203,180.655671,3.766837,225.422902,0.527984,8.653180,4.795684,5.064734,80.617892,1.310474
std,30.109286,172.573509,2.485410,212.247256,2.414489,15.428851,10.472271,5.650678,70.280483,0.906822
min,0.000000,0.000000,0.300000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.047059
25%,14.059701,72.906250,1.897059,90.774194,0.000000,0.000000,1.000000,1.000000,30.000000,0.675000
50%,29.860465,146.755435,3.210084,184.666667,0.000000,4.000000,3.000000,3.000000,60.000000,1.108911
75%,48.482558,237.268750,4.967213,298.125000,0.000000,10.000000,5.000000,7.000000,110.000000,1.703704
max,433.533333,2830.733333,27.472222,3413.533333,76.000000,287.000000,238.000000,51.000000,564.000000,11.488372


3. Record the major update dates of the iOS/Android system in Excel, e.g., iOS 14.3 was released on Dec 14th, 2020. This information is helpful as we want to see if apps update differently when there is a major upgrade in the operating system. One of our referees is worried that iOS system upgrades cause the effect of privacy labels we documented, so we need to take a closer look.

4. Plot heat map of version updates for type 2 lists. You can use calplot or other packages that you find suitable. Plot the heatmap of all/major/minor updates for the iOS and Android apps separately. E.g., major updates of iOS (one subgraph) and Android (another subgraph), then put both subgraphs in one graph. You can do this for the whole sample in type 2 lists first and then focus on individual apps (popular ones with "rank" <= 50); make sure the sample is paired in the sense that the Android and iOS versions of the same app(s) are used. This is to facilitate comparisons for before vs after privacy label policy and iOS vs Android and detect any patterns visually.